# Challenge 5: Multi-Source, Multi-Agent
## Introduction

In this part of the challenge you will add another source of data (structured) to the solution.

### Importing the libraries

In [10]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### Create Environment Variables

**Important:** Make sure you update your `.env` file.

In [11]:
import os, dotenv,sys
dotenv.load_dotenv()

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))


# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")

# Local Folder for the documents
LOCAL_FOLDER = "D:\fsi2023"

# SQL Database
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DRIVER = os.getenv("SQL_DRIVER")

Make Sure you have the ODBC driver installed

In [12]:
# Check the installed ODBC drivers (you should have installed the "ODBC Driver 18 for SQL Server")
import pyodbc

print(pyodbc.drivers())
assert os.getenv('SQL_DRIVER') in pyodbc.drivers()

['SQL Server', 'ODBC Driver 18 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


## Step 1: Import data to the SQL Database

The database provided is an Azure SQL Database.
Upload the data to the SQL Database using the `./data/fsi/db/create_stock_table.sql` script.

The script will create a table named "stock" in the provided database.
The table contains the following columns:
- **Date** DATE
- **CloseLast** DECIMAL(10, 2) 
- **Volume** INT
- **Open** DECIMAL(10, 2)
- **High** DECIMAL(10, 2)
- **Low** DECIMAL(10, 2)
- **Symbol** VARCHAR(10)

In [13]:
# Create the Connection to the SQL Server Database (sqlalchemy)
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

connection_string = f"Driver={SQL_DRIVER};Server=tcp:{SQL_SERVER},1433;Database={SQL_DB};Uid={SQL_USERNAME};Pwd={SQL_PWD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
quoted_conn_str = quote_plus(connection_string)
# Create the sqlalchemy engine (mssql+pyodbc)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted_conn_str))


In [ ]:
# Create the Stock Table in the SQL Server Database
with open('../../data/fsi/db/create_stock_table.sql', 'r') as file:
    sql_statements = file.read()

# Execute the SQL Statements
with engine.connect() as connection:
    for command in sql_statements.split('GO'):
        command = command.strip()
        if command:
            connection.execute(text(command))
    connection.execute(text("commit"))

## Step 2: Create a LangChain SQL Agent

In [14]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(api_key = AZURE_OPENAI_API_KEY,  
                      api_version = "2024-06-01",
                      azure_endpoint = AZURE_OPENAI_ENDPOINT,
                      model= AZURE_OPENAI_DEPLOYMENT_NAME,
                      streaming=True)

In [15]:
# Create the LangChain SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
db = SQLDatabase(engine=engine)
stock_toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [16]:
# Create the agent using the Langhcain SQL Agent Class (create_sql_agent)
stock_agent = create_sql_agent(
    toolkit=stock_toolkit,
    llm=llm,
    agent_type="openai-tools",
    agent_name="StockAgent",
    agent_description="Stock Agent",
    agent_version="0.1",
    agent_author="itsarag",
    #verbose=True,
    agent_executor_kwargs=dict(handle_parsing_errors=True))

In [17]:
# Structure the final prompt from the ChatPromptTemplate
# include the system message to the user message
# System message: You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices. If you can't find the answer, say 'I am unable to find the answer.'
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
          You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices.
          If you can't find the answer, say 'I am unable to find the answer.'
         """
         ),
        ("user", "{question}\n ai: "),
    ]
)

In [18]:
# Test the agent
response = stock_agent.invoke(final_prompt.format(question="What was the price of Apple,Nvidia and Microsoft stock in 23/07/2024?"))
print(response['output'])

On 23/07/2024, the closing prices for the stocks were as follows:

- Apple (AAPL): $225.01
- Nvidia (NVDA): $122.59
- Microsoft (MSFT): $444.85


## Step 3: Create the Multi Agent Solution

In [43]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

## Import the necessary libraries

from typing import Annotated, Sequence
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langgraph.graph import StateGraph, END
from urllib.parse import quote_plus 
from its_a_rag import ingestion
from sqlalchemy import create_engine
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate

In [44]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(api_key = AZURE_OPENAI_API_KEY,  
                      api_version = "2024-06-01",
                      azure_endpoint = AZURE_OPENAI_ENDPOINT,
                      model= AZURE_OPENAI_DEPLOYMENT_NAME,
                      streaming=True)

In [45]:
# Create the Agent State Class to store the messages between the agents
class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [46]:
# Create the start_agent that analyze the user question and decide if the question is related to stock prices or financial results
def start_agent(state):
    # Import the global llm
    global llm
    start_agent_llm = llm
    # Prepare the prompt for the agent
    # Prompt Example: You are an agent that needs analyze the user question. \n Question : {input} \n if the question is related to stock prices answer with "stock". \n if the question is related to information about financial results answer with "rag". \n if the question is unclear or you cannot decide answer with "rag". \n only answer with one of the word provided. Your answer (stock/rag):
    prompt = PromptTemplate.from_template("""
    You are an agent that needs analyze the user question. \n
    Question : {input} \n
    if the question is related to stock prices answer with "stock". \n
    if the question is related to information about financial results answer with "rag". \n
    if the question is unclear or you cannot decide answer with "rag". \n
    only answer with one of the word provided.
    Your answer (stock/rag):
    """)
    # Prepare the chain to be executed
    chain = prompt | start_agent_llm
    # invoke the chain
    response = chain.invoke({"input": state["input"]})
    # take the decision from the response
    decision = response.content.strip().lower()
    # Return the response for the next agent (decision and input required coming fron the Agent State)
    return {"decision": decision, "input": state["input"]}

In [47]:
## Stock Agent

def stock_agent(state):
    # Import the LLM
    global llm
    stock_agent_llm = llm
    # Create the DB Connection
    connection_string = f"Driver={SQL_DRIVER};Server=tcp:{SQL_SERVER},1433;Database={SQL_DB};Uid={SQL_USERNAME};Pwd={SQL_PWD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
    quoted_conn_str = quote_plus(connection_string)
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted_conn_str))
    # Create the SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
    db = SQLDatabase(engine=engine)
    stock_toolkit = SQLDatabaseToolkit(db=db, llm=stock_agent_llm)
    # Create the agent using the Langhcain SQL Agent Class (create_sql_agent)
    stock_agent = create_sql_agent(
        toolkit=stock_toolkit,
        llm=stock_agent_llm,
        agent_type="openai-tools",
        agent_name="StockAgent",
        agent_description="Stock Agent",
        agent_version="0.1",
        agent_author="itsarag",
        #verbose=True,
        agent_executor_kwargs=dict(handle_parsing_errors=True))
    # Structure the final prompt from the ChatPromptTemplate
    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", 
            """
            You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices. \n
            If you can't find the answer, say 'I am unable to find the answer.'
            """
            ),
            ("user", "{question}\n ai: "),
        ]
    )
    # Prepare the response using the invoke method of the agent
    response = stock_agent.invoke(final_prompt.format(question=state["input"]))
    # Return the response for the next agent (output and input required coming fron the Agent State)
    return {"output": response["output"], "input": state["input"]}

In [48]:
## Node rag (this is a clean implementation of the RAG Agent completed in Challenge 4)

def rag_agent(state):
    # Define the LLM
    global llm
    rag_agent_llm = llm
    # Define the index (use the one created in the previous challenge)
    retriever_multimodal = AzureAISearchRetriever(index_name=AZURE_SEARCH_INDEX, api_key=AZURE_SEARCH_API_KEY, service_name=AZURE_SEARCH_ENDPOINT, top_k=5)
    # Define the chain (as it was in the previous challenge)
    chain_multimodal_rag = (
    {
        "context": retriever_multimodal | RunnableLambda(ingestion.get_image_description),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(ingestion.multimodal_prompt)
    | rag_agent_llm
    | StrOutputParser()
    )
    # prepare the response using the invoke method of the agent
    response = chain_multimodal_rag.invoke({"input": state["input"]})
    # Return the response for the next agent (output and input required coming from the Agent State)
    return {"output": response, "input": state["input"]}

In [49]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str

# Create the 3 steps graph that is going to be working in the bellow "decision" condition
# Add nodes (start_agent, stock_agent, rag_agent) and conditional edges where the decision with be stock or rag
def create_graph():
    # Create the Workflow as StateGraph using the AgentState
    workflow = StateGraph(AgentState)
    # Add the nodes (start_agent, stock_agent, rag_agent)
    workflow.add_node("start", start_agent)
    workflow.add_node("stock_agent", stock_agent)
    workflow.add_node("rag_agent", rag_agent)
    # Add the conditional edge from start -> lamba (decision) -> stock_agent or rag_agent
    workflow.add_conditional_edges(
        "start",
        lambda x: x["decision"],
        {
            "stock": "stock_agent",
            "rag": "rag_agent"
        }
    )
    # Set the workflow entry point
    workflow.set_entry_point("start")
    # Add the final edges to the END node
    workflow.add_edge("stock_agent", END)
    workflow.add_edge("rag_agent", END)
    #Compile the workflow
    return workflow.compile()

### Step 4: Test the Solution

In [50]:
## Test Solution

# intantiate the graph (create_graph)
graph = create_graph()

# Use the graph invoke to answer the questions
# Test the graph with various questions
questions = [
    "What are the revenues of Google in the year 2000?",
    "What are the revenues and the operative margins of Alphabet Inc. in 2022 and how it compares with the previous year?",
    "Can you compare and create a table with the revenue of Alphabet Inc., NVIDIA, MICROSOFT, APPLE and AMAZON in years 2023?",
    "Did APPLE repurchase common stock in 2023? create a table of Apple repurchased stock with date, numbers of stocks and values in dollars.",
    "Can you give me the Fiscal Year 2023 Highlights for Apple, Microsoft, Nvidia and Google?"
]

for question in questions:
    result = graph.invoke({"input": question})
    print(result["output"])


On 23/07/2024, the closing prices for the stocks were as follows:
- Apple (AAPL): $225.01
- Nvidia (NVDA): $122.59
- Microsoft (MSFT): $444.85
I don't know.
I don't have the revenue data for Intel for the years 2019 to 2023 in the provided context.
I don't know. The provided context does not include information about the Intel CEO in 2023.
